# **Kaggle – DataTops®**
Tu TA ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a tu profe a mejorar ese modelo?

## Aspectos importantes
- Última submission:
    - Mañana: 17 de febrero a las 5pm
    - Tarde: 19 de febrero a las 5pm
- **Enlace de la competición**: https://www.kaggle.com/t/c5cc87b50c4b4770bdc8f5acbe15577d
- **Requisito**: Estar registrado en [Kaggle](https://www.kaggle.com/)

## Métrica:
El error cuadrático medio (RMSE, por sus siglas en inglés) es una medida de la desviación estándar de los residuos (errores de predicción). Los residuos representan la diferencia entre los valores observados y los valores predichos por el modelo. El RMSE indica qué tan dispersos están estos errores: cuanto menor es el RMSE, más cercanas están las predicciones a los valores reales. En otras palabras, el RMSE mide qué tan bien se ajusta la línea de regresión a los datos.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## 1. Librerías

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import urllib.request
import matplotlib.pyplot as plt

## 2. Datos

In [ ]:
df = pd.read_csv("./data/train.csv")



In [ ]:
train = pd.read_csv("./data/train.csv")
test  = pd.read_csv("./data/test.csv")
sample_sub = pd.read_csv("./data/sample_submission.csv")

print("train:", train.shape)
print("test:", test.shape)
print("sample:", sample_sub.shape)
train.head()


### 2.1 Exploración de los datos

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

In [ ]:
df.isna().sum().sort_values(ascending=False).head(20)



In [ ]:
df.duplicated().sum()


In [ ]:
df.describe(include="all").T


In [ ]:
target = "Price_in_euros"

df[target].describe()


In [ ]:
plt.figure(figsize=(6,4))
plt.hist(df[target], bins=30)
plt.title("Distribución de Price_in_euros")
plt.xlabel("Price_in_euros")
plt.ylabel("Frecuencia")
plt.show()


In [ ]:
#  Aquí solo creamos columnas más “analizables” (numéricas). No borramos las originales.
df_eda = df.copy()

# RAM: "8GB" -> 8
df_eda["Ram_GB"] = df_eda["Ram"].str.replace("GB", "", regex=False).astype(float)

# Weight: "1.86kg" -> 1.86
df_eda["Weight_kg"] = df_eda["Weight"].str.replace("kg", "", regex=False).astype(float)

# ScreenResolution: extraer ancho y alto
wh = df_eda["ScreenResolution"].str.extract(r'(?P<width>\d+)\s*x\s*(?P<height>\d+)')
df_eda["Screen_Width"]  = wh["width"].astype(float)
df_eda["Screen_Height"] = wh["height"].astype(float)

# PPI
df_eda["PPI"] = np.sqrt(df_eda["Screen_Width"]**2 + df_eda["Screen_Height"]**2) / df_eda["Inches"]

# Flags
df_eda["Touchscreen"] = df_eda["ScreenResolution"].str.contains("Touchscreen", case=False, na=False).astype(int)
df_eda["IPS_Panel"]   = df_eda["ScreenResolution"].str.contains("IPS", case=False, na=False).astype(int)

# CPU: marca y GHz
df_eda["Cpu_Brand"] = df_eda["Cpu"].str.split().str[0]
df_eda["Cpu_GHz"] = df_eda["Cpu"].str.extract(r'(\d+(\.\d+)?)\s*GHz')[0].astype(float)

# GPU: marca
df_eda["Gpu_Brand"] = df_eda["Gpu"].str.split().str[0]

df_eda[["Ram", "Ram_GB", "Weight", "Weight_kg", "ScreenResolution", "PPI", "Cpu", "Cpu_GHz"]].head()


In [ ]:
# Correlación con el target (solo numéricas)
num_cols = df_eda.select_dtypes(include=[np.number]).columns

corr_target = df_eda[num_cols].corr()[target].drop(target)

# Ordenar por valor absoluto 
corr_target = corr_target.reindex(corr_target.abs().sort_values(ascending=False).index)

corr_target.head(15)


In [ ]:
# Colinealidad usando correlacion entre Features
corr_matrix = df_eda[num_cols].corr().abs()

plt.figure(figsize=(10,7))
plt.imshow(corr_matrix, aspect="auto")
plt.colorbar()
plt.xticks(range(len(num_cols)), num_cols, rotation=90)
plt.yticks(range(len(num_cols)), num_cols)
plt.title("Correlación absoluta entre variables numéricas")
plt.show()


In [ ]:
# Ver variables categóricas
cat_cols = df.select_dtypes(include="object").columns

cat_cols



In [ ]:
df_corr = df_eda.copy()


In [ ]:
# Matriz de correlación SOLO numérica
num_cols = df_corr.select_dtypes(include=np.number).columns

corr_matrix = df_corr[num_cols].corr()


In [ ]:
plt.figure(figsize=(10,8))

plt.imshow(corr_matrix, aspect="auto")
plt.colorbar()

plt.xticks(range(len(num_cols)), num_cols, rotation=90)
plt.yticks(range(len(num_cols)), num_cols)

plt.title("Matriz de correlación (limpia)")

plt.show()


In [ ]:
# correlación entre variables numéricas y el target 
num_cols = df_corr.select_dtypes(include=np.number).columns


In [ ]:
corr_target = df_corr[num_cols].corr()[target]

# quitar el propio target
corr_target = corr_target.drop(target)

# ordenar por importancia (valor absoluto)
corr_target = corr_target.reindex(
    corr_target.abs().sort_values(ascending=False).index
)

corr_target.head(15)


In [ ]:
plt.figure(figsize=(8,6))

plt.barh(corr_target.index[:15], corr_target.values[:15])

plt.title("Variables más correlacionadas con el precio")
plt.xlabel("Correlación")
plt.gca().invert_yaxis()

plt.show()


### 2.3 Definir X e y

In [ ]:
# Definir target
target = "Price_in_euros"

# X = todas las columnas excepto el target
X = df.drop(columns=[target])

# y = target
y = df[target].copy()

print("Shape X:", X.shape)
print("Shape y:", y.shape)


### 2.4 Dividir X_train, X_test, y_train, y_test

In [ ]:
from sklearn.model_selection import train_test_split

# División train / test (validación interna)
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,   # 20% para test interno
    random_state=42
)

print("X_train:", X_train.shape)
print("X_test :", X_test.shape)
print("y_train:", y_train.shape)
print("y_test :", y_test.shape)


In [ ]:
def feature_engineering(df):

    df = df.copy()

    # RAM: "8GB" -> 8
    df["Ram_GB"] = df["Ram"].str.replace("GB","", regex=False).astype(float)

    # Weight: "1.86kg" -> 1.86
    df["Weight_kg"] = df["Weight"].str.replace("kg","", regex=False).astype(float)

    # ScreenResolution -> width y height
    wh = df["ScreenResolution"].str.extract(r'(?P<width>\d+)\s*x\s*(?P<height>\d+)')

    df["Screen_Width"] = wh["width"].astype(float)
    df["Screen_Height"] = wh["height"].astype(float)

    # PPI
    df["PPI"] = np.sqrt(df["Screen_Width"]**2 + df["Screen_Height"]**2) / df["Inches"]

    # Flags
    df["Touchscreen"] = df["ScreenResolution"].str.contains("Touchscreen", case=False, na=False).astype(int)
    df["IPS_Panel"] = df["ScreenResolution"].str.contains("IPS", case=False, na=False).astype(int)

    # CPU GHz
    df["Cpu_GHz"] = df["Cpu"].str.extract(r'(\d+(\.\d+)?)\s*GHz')[0].astype(float)

    return df


In [ ]:
# Feature Engineering

# Aplicar FE SOLO después del split

X_train_fe = feature_engineering(X_train)
X_test_fe  = feature_engineering(X_test)


In [ ]:
# aplicar get_dummies a train y test
X_train_encoded = pd.get_dummies(X_train_fe, drop_first=True)
X_test_encoded  = pd.get_dummies(X_test_fe, drop_first=True)


In [ ]:
# alinear columnas
X_train_encoded, X_test_encoded = X_train_encoded.align(
    X_test_encoded,
    join="left",
    axis=1,
    fill_value=0
)


In [ ]:
print("X_train_encoded:", X_train_encoded.shape)
print("X_test_encoded :", X_test_encoded.shape)

print("Columnas iguales:", (X_train_encoded.columns == X_test_encoded.columns).all())


Se aplica OneHotEncoding a las variables categóricas. Posteriormente se alinean las columnas entre train y test para asegurar que ambos conjuntos tengan exactamente las mismas features.

## 3. Procesado de datos

Nuestro target es la columna `Price_in_euros`

-----------------------------------------------------------------------------------------------------------------

## 4. Modelado

### 4.1 Baseline de modelos


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

# Crear modelo baseline
rf_model = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

# Entrenar
rf_model.fit(X_train_encoded, y_train)

# Predicciones en test interno
predictions = rf_model.predict(X_test_encoded)

# Métrica RMSE
rmse = root_mean_squared_error(y_test, predictions)

print("RMSE baseline:", rmse)


### 4.2 Sacar métricas, valorar los modelos

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

In [ ]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

# Calcular métricas

rmse = root_mean_squared_error(y_test, predictions)
mae  = mean_absolute_error(y_test, predictions)
r2   = r2_score(y_test, predictions)

print("RMSE:", rmse)
print("MAE :", mae)
print("R2  :", r2)


### 4.3 Optimización (up to you 🫰🏻)

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

## 1. Carga los datos de `test.csv` para predecir.


In [ ]:
X_pred = pd.read_csv("./data/test.csv")
X_pred.head()

In [ ]:
X_pred.tail()

In [ ]:
X_pred.info()

 ## 2. Replicar el procesado para ``test.csv``

In [ ]:
X_pred_fe = feature_engineering(test)


In [ ]:
# 1) aplicar FE al test
X_pred_fe = feature_engineering(test)

# 2) dummies
X_pred = pd.get_dummies(X_pred_fe, drop_first=True)

# 3) alinear con train
X_pred_aligned = X_pred.reindex(columns=X_train_encoded.columns, fill_value=0)

# 4) predecir
predictions_submit = rf_model.predict(X_pred_aligned)

len(predictions_submit)


In [ ]:
X_pred

In [ ]:
predictions_submit = rf_model.predict(X_pred_aligned)
predictions_submit[:5]


In [ ]:
len(predictions_submit)


**¡OJO! ¿Por qué me da error?**

IMPORTANTE:

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO **BORRAR FILAS**, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER **391 FILAS**, SI O SI

**Entonces, si al cargar los datos de ``train.csv`` usaste `index_col=0`, ¿tendré que hacer lo también para el `test.csv`?**

In [ ]:
# ¿Qué opináis?
# ¿Sí, no?.

![wow.jpeg](attachment:wow.jpeg)

## 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`.

In [ ]:
# 1) reconstruir sample limpio (variable que usa el chequeador)
sample = pd.read_csv("./data/sample_submission.csv")

# 2) reconstruir submission desde la plantilla
submission = sample.copy()

# 3) meter predicciones (asegúrate de que predictions_submit existe y tiene 391 valores)
price_col = [c for c in submission.columns if "price" in c.lower()][0]
submission[price_col] = predictions_submit

print("sample shape:", sample.shape)
print("submission shape:", submission.shape)
print("IDs iguales:", submission["laptop_ID"].equals(sample["laptop_ID"]))
submission.head()


In [ ]:
sample = pd.read_csv("./data/sample_submission.csv")  # IMPORTANTÍSIMO: el chequeador usa 'sample'
print("sample shape:", sample.shape)
print(sample.head())


In [ ]:
submission = sample.copy()

price_col = [c for c in submission.columns if "price" in c.lower()][0]
submission[price_col] = predictions_submit

print("submission shape:", submission.shape)
print(submission.head())


## 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [ ]:
#¿Cómo creamos la submission?
submission = pd.DataFrame()

In [ ]:
submission.head()

In [ ]:
submission.shape

In [ ]:
print("sample shape:", sample.shape)
print("submission shape:", submission.shape)

print(sample.columns)
print(submission.columns)


## 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.

    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.

    Si no, LEE EL MENSAJE Y HAZLE CASO.

    Si aún no:
    - apaga tu ordenador,
    - date una vuelta,
    - enciendelo otra vez,
    - abre este notebook y
    - leelo todo de nuevo.
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                df_to_submit.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")
                img = Image.open("gfg.png")
                img.show()
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [ ]:
chequeador(submission)